# Hente hemmeligheter fra secret manager

In [ ]:
from google.cloud import secretmanager
from google.oauth2 import service_account

In [ ]:
credentials = service_account.Credentials.from_service_account_file('/home/jupyter/sa_key.json')

In [ ]:
secrets = secretmanager.SecretManagerServiceClient(credentials=credentials)

In [ ]:
project_name = "knada-gcp"
secret_name = "vebjorn-rekkebo-bac1"
version = "latest"

resource_name = f"projects/{project_name}/secrets/{secret_name}/versions/{version}"
secret = secrets.access_secret_version(name=resource_name)

In [ ]:
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

In [ ]:
import psycopg2

import pandas as pd
import datetime

from sqlalchemy import create_engine

In [ ]:
method = "postgresql+psycopg2"
host = "A01DBVL028.adeo.no:5432"
#host = "10.53.15.222:5432"
db_name = "pensjon-psak"

alchemyEngine = create_engine(f'{method}://{secrets["PSAK_POSTGRES_USER"]}:{secrets["PSAK_POSTGRES_PASSWORD"]}@{host}/{db_name}')
dbConnection = alchemyEngine.connect()

# Div utforsking

In [ ]:
df_tables = pd.read_sql(
    """
    select * from information_schema.tables where table_schema='public';
    """,
    dbConnection)

In [ ]:
df_tables

In [ ]:
df_pref = pd.read_sql(
    """
    select * from information_schema.columns where table_name='t_saksbeh_preferanser';
    """,
    dbConnection)

In [ ]:
df_spring = pd.read_sql(
    """
    select * from spring_session;
    """,
    dbConnection)

In [ ]:
df_last_session = pd.read_sql(
    """
    select distinct on (principal_name) *
    from spring_session
    order by principal_name, last_access_time desc;
    """,
    dbConnection)

In [ ]:
df_last_session["last_access_datetime"] = df_last_session.last_access_time.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))

In [ ]:
import plotly.express as px

px.scatter(df_last_session, "last_access_datetime")

In [ ]:
df_pref.head()

In [ ]:
df_endret = pd.read_sql(
    """
    select count(*) from t_saksbeh_preferanser
    where dato_endret=(select min(dato_endret) from t_saksbeh_preferanser);
    """,
    dbConnection)

In [ ]:
df_endret